In [1]:
import itk

In [2]:
# ----------------------
# Lecture de l'image
# ----------------------
input_filename = 'data/brain.png'
image = itk.imread(input_filename)
ImageType = type(image) # On récupère le type de l'image
# ----------------------
# Création et paramétrage du filtre
# ----------------------
# On créé une instance du filtre qui prend en entrée et ressort des images du même type que l'image d'entrée
smoothFilter = itk.SmoothingRecursiveGaussianImageFilter[ImageType,ImageType].New() 
smoothFilter.SetInput(image) # On spécifie l'image d'entrée du filtre

In [4]:
sigma = 5
smoothFilter.SetSigma(sigma) # On sélectionne la variance du filtre
# ----------------------
# Écriture de la sortie du filtre
# ----------------------
# On écrit sur le disque la sortie du filtre Gaussien
itk.imwrite(smoothFilter.GetOutput(), 'output/myImage_smoothed.png') 

In [5]:
absolutFilter = itk.AbsoluteValueDifferenceImageFilter[ImageType,ImageType,ImageType].New()
absolutFilter.SetInput1(image)
absolutFilter.SetInput2(smoothFilter.GetOutput())
itk.imwrite(absolutFilter.GetOutput(), 'output/myImage_absolut.png')

### Exercice 2

In [6]:
fixed_image = itk.imread('data/BrainProtonDensitySliceBorder20.png')
moving_image = itk.imread('data/BrainProtonDensitySliceShifted13x17y.png')

In [7]:
optimizer = itk.RegularStepGradientDescentOptimizer.New() # Instance de laclasse d'optimiseur choisie
optimizer.SetMaximumStepLength(10) # Borne supérieure du pas de descente (en pixel)
optimizer.SetMinimumStepLength(1) # Borne inférieure du pas de descente (en pixel)
optimizer.SetNumberOfIterations(50) # Nombre maximal d'itération

In [9]:
initialTransform = itk.TranslationTransform[itk.D, 2].New() # Instance de la classe de transformation choisie
initialParameters = initialTransform.GetParameters() # Récupération des paramètres de la transformation
initialParameters[0] = 13 # Initialisation du paramètre de translation en x
initialParameters[1] = -17 # Initialisation du paramètre de translation en y

In [11]:
registration_filter = itk.ImageRegistrationMethod[ImageType, ImageType].New() # Instance de la classe de recalage choisie
registration_filter.SetFixedImage(fixed_image) # Image de référence
registration_filter.SetMovingImage(moving_image) # Image à recaler
registration_filter.SetOptimizer(optimizer)
registration_filter.SetTransform(initialTransform) # Transformation
registration_filter.SetInitialTransformParameters(initialParameters) # Application de la transformation initiale
registration_filter.SetInterpolator(itk.LinearInterpolateImageFunction[ImageType, itk.D].New()) # Interpolateur
registration_filter.SetMetric(itk.MeanSquaresImageToImageMetric[ImageType, ImageType].New()) # Métrique
registration_filter.Update() # Exécution du recalage

In [12]:
final_transform = registration_filter.GetTransform()

In [13]:
resample_filter = itk.ResampleImageFilter[ImageType,ImageType].New() # Instance de la classe de ré-échantillonnage
resample_filter.SetInput(moving_image) # Image d'entrée
resample_filter.SetTransform(final_transform)
resample_filter.SetSize(fixed_image.GetLargestPossibleRegion().GetSize())
itk.imwrite(resample_filter.GetOutput(), 'output/image_recalee.png') # Ecriture de l'image recalée sur le disque